In [1]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.5/532.5 kB 14.9 MB/s eta 0:00:00


In [2]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.2/389.2 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/

In [3]:
import json
import torch
from datasets import Dataset, load_dataset
from peft import LoraConfig
from transformers import AutoTokenizer, TextStreamer

BASE_SAMPLES = [
    {
        "dealer_name": "CHARBHUJA TRACTOR & COMPRESSOR",
        "model_name": "POWERTRAC EURO 42 PLUS",
        "horse_power": 42,
        "asset_cost": 800000,
        "raw_text": """CHARBHUJA TRACTOR & COMPRESSOR
ESCORTS POWERTRAC TRACTORS
Near Sharma Hospital KANKROLI
Quotation No 426
POWERTRAC EURO 42 PLUS
42 HP
Amount Rs 8,00,000"""
    },
    {
        "dealer_name": "AUTO UNION JODHPUR",
        "model_name": "ME/TAFE TRACTOR ME 241",
        "horse_power": 42,
        "asset_cost": 590000,
        "raw_text": """MASSEY FERGUSON
AUTO UNION JODHPUR
ME/TAFE TRACTOR ME 241
(42 HP)
TOTAL 590000"""
    },
    {
        "dealer_name": "SHREE BALAJI TRACTORS",
        "model_name": "SWARAJ 744 FE",
        "horse_power": 48,
        "asset_cost": 720000,
        "raw_text": """SHREE BALAJI TRACTORS
AUTH DEALER SWARAJ
MODEL SWARAJ 744 FE
48 HP
PRICE 7,20,000"""
    },
    {
        "dealer_name": "MAHINDRA TRACTORS",
        "model_name": "MAHINDRA 275 DI TU",
        "horse_power": 39,
        "asset_cost": 650000,
        "raw_text": """MAHINDRA & MAHINDRA LTD
MAHINDRA 275 DI TU
39 HP
TOTAL AMOUNT 6,50,000"""
    }
]
import random
import re

def add_ocr_noise(text):
    replacements = {
        "O": "0",
        "I": "1",
        "S": "5",
        "E": "F",
        "&": "AND",
        "HP": random.choice(["HP", "H P", "H.P", "H P."]),
    }

    for k, v in replacements.items():
        if random.random() < 0.3:
            text = text.replace(k, v)

    # Random casing
    if random.random() < 0.4:
        text = text.lower()
    elif random.random() < 0.4:
        text = text.upper()

    # Random spacing noise
    text = re.sub(r"\s+", lambda m: " " * random.randint(1, 3), text)

    # Random numeric separators
    text = text.replace("800000", random.choice(["8,00,000", "800000", "8.00.000"]))
    text = text.replace("590000", random.choice(["5,90,000", "590000"]))

    return text
def build_prompt(raw_text):
    return (
        "Extract the following fields as JSON from this noisy OCR text of a tractor quotation: "
        "dealer_name, model_name, horse_power, asset_cost. "
        "Return only valid JSON, use null for missing values.\n\n"
        f"Raw text:\n{raw_text}"
    )


def build_completion(sample):
    return json.dumps({
        "dealer_name": sample["dealer_name"],
        "model_name": sample["model_name"],
        "horse_power": sample["horse_power"],
        "asset_cost": sample["asset_cost"]
    })
def generate_synthetic_data(n_samples=100):
    data = []

    for i in range(n_samples):
        base = BASE_SAMPLES[i % len(BASE_SAMPLES)]

        noisy_text = add_ocr_noise(base["raw_text"])

        data.append({
            "prompt": build_prompt(noisy_text),
            "completion": build_completion(base)
        })

    return data


In [4]:
synthetic_data = generate_synthetic_data(100)

with open("tractor_data.jsonl", "w") as f:
    for row in synthetic_data:
        f.write(json.dumps(row) + "\n")


In [5]:
from trl import SFTTrainer
from unsloth import FastLanguageModel


/tmp/ipython-input-841359825.py:2: UserWarning: WARNING: Unsloth should be imported before [trl, transformers, peft] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [18]:
from trl import SFTConfig

# Load dataset
dataset = load_dataset("json", data_files="tractor_data.jsonl", split="train")

# Step 2: Load Base Model with 4-bit Quantization
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    max_seq_length=max_seq_length,
    dtype=None,  # Auto-detect
    load_in_4bit=True,
)

# Step 3: Configure LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

# Step 4: Fine-Tune
from transformers import TrainingArguments

# ─────────────────────────────────────────────────────────────
# Pre-format dataset → avoid TRL's automatic EOS / formatting issues
# ─────────────────────────────────────────────────────────────

def format_for_training(example):
    text = f"{example['prompt']}{tokenizer.eos_token}{example['completion']}"
    return {"text": text}

# Apply formatting once, before creating trainer
formatted_dataset = dataset.map(
    format_for_training,
    num_proc=2,           # safe on Colab T4
    desc="Formatting dataset with prompt + completion + EOS"
)

# ─────────────────────────────────────────────────────────────
# Now train with dataset_text_field instead of formatting_func
# ─────────────────────────────────────────────────────────────

from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,

    dataset_text_field="text",          # ← tell trainer where the full text is

    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,                   # testing → change to num_train_epochs=3 later
        # num_train_epochs=3,
        learning_rate=2e-4,
        fp16=True,
        bf16=False,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",

        # Important: keep this false since we're pre-formatting
        completion_only_loss=False,
    ),
)

trainer.train()

model.save_pretrained("custom_phi3_tractor_extractor")         # saves LoRA adapters + config
tokenizer.save_pretrained("custom_phi3_tractor_extractor")

# Optional: merge LoRA into base model (16-bit) for easier inference later
model.save_pretrained_merged(
    "custom_phi3_tractor_extractor_merged",
    tokenizer,
    save_method="merged_16bit"   # or "merged_4bit" if you want smaller size
)

==((====))==  Unsloth 2026.1.3: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Formatting dataset with prompt + completion + EOS (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files:  50%|█████     | 1/2 [02:28<02:28, 148.00s/it]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [05:12<00:00, 156.32s/it]


Unsloth: Merge process complete. Saved to `/content/custom_phi3_tractor_extractor_merged`


In [19]:
# Step 6: Load the Model and Infer
# Reload (simulating separate run)
model, tokenizer = FastLanguageModel.from_pretrained(
    "custom_phi3_tractor_extractor_merged",  # Or original + adapters
    dtype=None,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)  # Enables faster inference



==((====))==  Unsloth 2026.1.3: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32009)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Llam

In [23]:
# ─────────────────────────────────────────────────────────────
# Fixed Inference: Get ONLY the generated JSON (no prompt repeat)
# ─────────────────────────────────────────────────────────────

from transformers import TextStreamer
import torch

# Your sample raw OCR (unchanged)
sample_raw = """GST - 08AAPPC9391K121\nQuotation\n* (02952) 222298\nMob. 9414172521\nCHARBHUJA TRACTOR & COMPRESSOR\n9929179791\nESCORTS\nAuth. Dealers : Escort Powertrac Tractors\nPOWERTRAC\nTRACTORS\nNear Sharma Hospital, TVS Chouraha, Nathdwara Road, KANKROLI - 313324\nNo.\n426\nDate.\n28|09|25\nM/s./Shri_\nDear Sir,\nunder :\nWe are pleased to submit our quotation for Tractor and Implementsyas per standard specification of the manufactures, as\nKAJSAMAMD\nDescription\nQty.\nPOWERTRAL CURD 42+\n1\nRate\n8,00,000\nAmount\n8,00,000-00\n45 MP\nHPH →\nIDFC FIRLT BANK 4O.\nTOTAL\n(Rs.)\n8,00,000-00\nTerms & Conditions:\nPrices:\nQuoted above are Ex-Godown Rajsamand, inclusive of taxex, duties and additional road transportation charges etc\nThe rates are strictly subjects to change without notice and shal be charged as ruling at the time of Delivery.\nDelivery Period\nSubject to Circumstances beyond our control\nPayment: Rs.\nsecurity deposit and balance full payment against delivery. Altematively. full payment by Demand Draft in\nadvance to be remitted to our Principals\nwhen demanded.\nTechnical Specifications: Manufacturers reserves the right to despatch the tractors from Faridabad or from any other assembly unit by road\nTransport to Rajsamand of the their\nmer Depot. The freight and other incidental charges are to be bome by the customer\nManufactures reserve the\nht to change any specification of the Tractors, Implements and accessones\nAn illustrate iterature of the\nTralcor and the Implements ordered by the customer is enclosed.\n13.00\nHopping to be favoured with valued orders\nThanking You!\nCustomer's Signature\nFor: CHARBHUJA TRACTOR & COMF\nsards niger"""

prompt_template = """Extract the following fields as JSON from this noisy OCR text of a tractor quotation: dealer_name, model_name, horse_power, asset_cost. Return only valid JSON, use null for missing values.

Raw text: {}"""

full_prompt = prompt_template.format(sample_raw)

# Tokenize prompt
inputs = tokenizer(full_prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

# Get length of prompt tokens (for slicing later)
input_length = inputs.input_ids.shape[1]

# Enable fast inference mode (Unsloth)
FastLanguageModel.for_inference(model)

# Streamer: skips prompt, streams only new tokens
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# Generate
outputs = model.generate(
    **inputs,
    max_new_tokens=128,          # enough for JSON
    do_sample=False,             # deterministic for debugging
    temperature=0.0,             # no randomness
    streamer=streamer,           # live stream ONLY new text
    eos_token_id=tokenizer.eos_token_id,
)

# Clean final output: slice only generated tokens + decode
generated_tokens = outputs[0, input_length:]  # everything after prompt
clean_output = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

print("\n\n=== Clean Generated Text (only model response) ===")
print(clean_output)

# If you want to force JSON parsing/debug (optional)
try:
    import json
    parsed = json.loads(clean_output)
    print("\nParsed JSON:", json.dumps(parsed, indent=2))
except json.JSONDecodeError as e:
    print("\nNot valid JSON yet:", e)

ia POWERTRAC TRACTORS
Model: POWERTRAC CURD 42+   HP 45  MF  Amount 8,00,000


=== Clean Generated Text (only model response) ===
ia POWERTRAC TRACTORS
Model: POWERTRAC CURD 42+   HP 45  MF  Amount 8,00,000

Not valid JSON yet: Expecting value: line 1 column 1 (char 0)
